In [1]:
from datasets import load_dataset
dataset = load_dataset("tdavidson/hate_speech_offensive")
print(dataset)

c:\Users\adity\Desktop\Study\hate_speech_detector\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['count', 'hate_speech_count', 'offensive_language_count', 'neither_count', 'class', 'tweet'],
        num_rows: 24783
    })
})


In [ ]:
dataset["train"].to_csv("train.csv")

In [6]:
import pandas as pd
import re
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [7]:
df = pd.read_csv('train.csv')

print("Class distribution:")
print(df['class'].value_counts())

def clean_tweet(tweet):
    if pd.isna(tweet):
        return ""
    tweet = re.sub(r'http\S+', ' URL ', tweet)
    tweet = re.sub(r'@\w+', ' USER ', tweet)
    tweet = re.sub(r'#(\w+)', r'HASHTAG_\1', tweet)    
    tweet = re.sub(r'\.{2,}', ' ELLIPSIS ', tweet)
    tweet = re.sub(r'!{2,}', ' EXCLAMATION ', tweet)    
    tweet = re.sub(r'[^\w\s!?*#$%&]', ' ', tweet)    
    tweet = re.sub(r'(.)\1{2,}', r'\1\1', tweet)    
    tweet = tweet.lower().strip()
    tweet = ' '.join(tweet.split())    
    return tweet

def minimal_clean(tweet):
    if pd.isna(tweet):
        return ""
    tweet = re.sub(r'http\S+', ' URL ', tweet)
    tweet = re.sub(r'@\w+', ' USER ', tweet)
    tweet = tweet.lower().strip()
    return ' '.join(tweet.split())

Class distribution:
class
1    19190
2     4163
0     1430
Name: count, dtype: int64


In [8]:
df['cleaned_tweet'] = df['tweet'].apply(minimal_clean)

X = df['cleaned_tweet']
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=10000,
        stop_words='english',
        ngram_range=(1, 3),
        min_df=2,
        max_df=0.9,
        analyzer='word',
        token_pattern=r'(?u)\b\w+\b',
    )),
    ('clf', RandomForestClassifier(
        n_estimators=200,
        random_state=42,
        n_jobs=-1,
        class_weight='balanced'
    ))
])

In [9]:
print("training model")
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
joblib.dump(pipeline, 'hate_speech_model.pkl')

training model

Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.15      0.23       286
           1       0.92      0.96      0.94      3838
           2       0.83      0.87      0.85       833

    accuracy                           0.90      4957
   macro avg       0.76      0.66      0.67      4957
weighted avg       0.88      0.90      0.88      4957



['hate_speech_model.pkl']